In [1]:
## 使用tablua包读取pdf表格
## 它的官网https://tabula-py.readthedocs.io/en/latest/
## 安装 
# !pip install tabula-py

In [2]:
import os, tabula, itertools
import numpy as np
import pandas as pd

In [3]:
def Str2TimeRange(timestr):
    if "-" in timestr:
        time_start, time_end = timestr.split('-')
        return list(pd.date_range(start=time_start, end=time_end, freq = 'D'))
    else:
        return pd.to_datetime(timestr, format="%Y/%m/%d")

In [4]:
PdfFilePath = "./E220809510428_出入境记录查询结果（电子文件）/48564c69e6d15eed649998176c4fa3de.pdf"
graduation_time = "2022-8-31"

In [5]:
# 每年需要更改这个部分
# 时间表示 YY/MM/DD

# 两个学期的上学时间
School_Time = ['2020/09/14-2021/01/12', '2021/01/13-2021/06/02', '2021/09/01-2022/01/11', '2022/01/12-2022/05/31']

# 第一学期 到 第四学期
School_Time = list(map(lambda x: Str2TimeRange(x), School_Time))
School_Time = [i for d in School_Time for i in d]

# 节假日（不包括暑假，上面学期已经排除暑假）当然这里加入暑假也是不冲突的
Holiday_Time = ['2020/10/1','2020/10/2','2020/10/25','2020/11/2', '2020/12/8', '2020/12/16-2020/12/27',\
    '2020/10/5','2020/10/26','2020/12/22','2020/12/31',\
    '2021/1/1','2021/2/3-2021/2/21','2021/4/2-2021/4/4','2021/5/1','2021/5/19','2021/6/14','2021/9/22',\
    '2021/10/1-2021/10/2', '2021/10/14','2021/11/2','2021/12/8','2021/12/17-2021/12/26',\
    '2021/2/11','2021/2/15-2021/2/16', '2021/4/5-2021/4/6','2021/5/3','2021/10/4','2021/12/27','2021/12/31',\
    '2022/1/1','2022/1/26-2022/2/13','2022/4/5','2022/4/15','2022/4/16','2022/5/1','2022/5/8','2022/6/3',\
    '2022/1/3','2022/1/31','2022/4/18','2022/5/2','2022/5/9']

Holiday_Time_sum = list(map(lambda x: Str2TimeRange(x), Holiday_Time))
Holiday_Time = []
for i in Holiday_Time_sum:
    if type(i) == list:
        Holiday_Time += i
    else:
        Holiday_Time.append(i)

In [6]:
# 使用tabula识别pdf的日期
# PdfFilePath 是放记录查询路径
df_list = tabula.read_pdf(PdfFilePath, pages = 'all', silent=True)

# 将读取的结果汇总一个list
df_sum = pd.DataFrame()
for df in df_list:
    if not len(df_sum):
        df_sum = df
    else:
        df_sum = pd.concat([df_sum, pd.DataFrame([list(df.columns)], columns = df_sum.columns)], axis = 0, ignore_index= True)

        df.columns = df_sum.columns
        df_sum = pd.concat([df_sum, df], axis = 0, ignore_index = True)

In [7]:
df_sum = df_sum.drop(['序号', '证件名称', '证件号码', '航班号'], axis = 1) 
df_sum['出入境日期'] = pd.to_datetime(df_sum['出入境日期'], format='%Y-%m-%d')
# 表格倒序排列
df_sum = df_sum.iloc[::-1, :] 
# 出入记录表读取好了

In [8]:
Total_time = pd.DataFrame(index = pd.date_range(start=School_Time[0], end = graduation_time, freq= 'D'), columns=['是否已经出境', '是否就读期间', '是否非周末', '是否非假期','排除节假日是否出境'])

# 是否已经出入境 NAN 表示已经入境
for start_index in range(0, len(df_sum), 2):
    start_time, end_time = df_sum.iloc[start_index, 1], df_sum.iloc[start_index+1, 1]
    if end_time.date() < School_Time[0].date() or start_time.date() > pd.to_datetime(graduation_time).date():
        continue
    start_time = max(start_time.date(), School_Time[0].date())
    end_time = min(end_time, pd.to_datetime(graduation_time))
    Total_time.loc[pd.date_range(start=start_time, end = end_time, freq= 'D'), '是否已经出境'] = True

# 是否就读期间 因为会有包括第二个学年结束 寒暑假 所以这里需要表示一下
Total_time.loc[School_Time, '是否就读期间'] = True

# 是否周末
Total_time.loc[list(map(lambda x: x.weekday() in [0,1,2,3,4],Total_time.index)),'是否非周末'] = True

# 是否非假日
Total_time.loc[Holiday_Time, '是否非假期'] = False

# 填充值
Total_time['是否非假期'] = Total_time['是否非假期'].fillna(True)
Total_time[['是否已经出境', '是否就读期间', '是否非周末']] = Total_time[['是否已经出境', '是否就读期间', '是否非周末']].fillna(False)

Total_time['排除节假日是否出境'] = Total_time[['是否已经出境', '是否就读期间', '是否非周末', '是否非假期']].all(axis='columns')

In [9]:
print("就读期间在澳门逗留总天数:{}".format(Total_time['是否已经出境'].sum()))
print("扣除周末、寒暑假及公共假期后在澳门逗留总天数:{}".format(Total_time['排除节假日是否出境'].sum()))

就读期间在澳门逗留总天数:308
扣除周末、寒暑假及公共假期后在澳门逗留总天数:208


In [10]:
def color_boolean(val):
    color =''
    if val == True:
        color = '#5FFE82'
    elif val == False:
        color = '#FE825F'
    return 'background-color: %s' % color
    
Total_time_style = Total_time.style.applymap(color_boolean)

Total_time_style.to_excel('record.xlsx', engine = 'openpyxl', encoding='utf-8-sig')